# Wesley trashnotebook

In [ ]:
# To install the packages and requirements for this notebook from the requirements.txt file
# Unhash the next line and run it. 

# !pip install -r requirements.txt

In [ ]:
# !brew cask install xquartz
# !brew install poppler antiword unrtf tesseract swig sox
# !pip3 install textract

In [ ]:
# !pip install pdfminer.six

In [ ]:
#!pip install chardet

In [ ]:
%run lib/__wesley_init_verbose__.py

In [ ]:
# This function returns the type of encoding in a pdf
# We want to use it to improve our decoding of pdfs
'''Make into a working function.
   Save the encoding type into the dataframe for later recall.
   '''

def determine_pdf_encoding(file):
    import chardet 
    rawdata = open(file, "rb").read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']
    return charenc

# Bing Search Scrape

Kelvin Christian's Bing Scraper with additionals from wesley init file. 

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re
import numpy as np
import pandas as pd
%run lib/__wesley_init__.py

# Declaring nessecary variables
base_web = "https://www.bing.com"
counter = 1
topic = input("Enter the topic: ").replace(" ", "+")
max_page = int(input("Enter tha maximum page scrape: "))
web_list = []
df = pd.DataFrame({'url':[], 'text':[]})


# Create the Spider class
class Bing_Spider(scrapy.Spider):
    name = "bing_spider"

    # start_requests method
    def start_requests(self):
        yield scrapy.Request(url = base_web + "/search?q=" + topic, callback = self.parse, dont_filter = True)

    # parse method
    def parse(self, response):

        global counter
        global web_list

        # get raw links in one page stored in a list
        web_list_this_page = response.xpath('//@href').extract()

        # website pattern for scraping
        pattern_http = re.compile("^https://")
        index_http = [bool(pattern_http.match(el)) for el in web_list_this_page]

        pattern_no_bing = re.compile("^(?!https://www.bing.com)")
        index_no_bing = [bool(pattern_no_bing.match(el)) for el in web_list_this_page]

        pattern_no_youtube = re.compile("^(?!https://www.youtube.com)")
        index_no_youtube = [bool(pattern_no_youtube.match(el)) for el in web_list_this_page]

        # intersection
        index = np.array(index_http) * np.array(index_no_bing) * np.array(index_no_youtube)

        # get clean links in a list
        web_list_this_page = list(np.array(web_list_this_page)[index])

        # concat the list with the previous page
        web_list += web_list_this_page

        # get link for the next page button
        next_page = response.xpath('//a[@class="sb_pagN sb_pagN_bp b_widePag sb_bp"]/@href').extract_first()

        # Output the number of page
        print("\n\nScraping page = "+str(counter))
        counter += 1

        # Check if this page is already maximum page
        if counter <= max_page:
            yield scrapy.Request(url = base_web + next_page, callback = self.parse)
        else:
            print("\n\nScraping done!")
            # Scraping texts
            for link in web_list:
                yield scrapy.Request(url = link, callback = self.parse_text)


    # parse texts in each links
    def parse_text(self, response):
        global df
        url = response.url
        text = " ".join(response.xpath('//text()').extract())
        df = pd.concat([df, pd.DataFrame({'url':[response.url], 'text':[text]})], axis = 0)


# Run the Spider
process = CrawlerProcess()
process.crawl(Bing_Spider)
process.start()


# Save to csv file
df.to_csv('ht_url_results2.csv')

In [ ]:
df.head()

# Full Preprocessing of Files to a csv

## Establish a list of the files in a directory


In [1]:
%run lib/__wesley_init__.py #Load functions from .py file in lib folder
import textract
from gensim.summarization import keywords
import warnings
warnings.filterwarnings("ignore")

#-----------------------------------------------------
#Initialize the data_log by discovering if it exists. If it does, load it. Otherwise, form one. 
def initialize_data_log():
    import pandas as pd
    topic = input('What topic are we looking at today?')
    topic = topic.rstrip()
    topic = topic.replace(' ', '_')
    answer = find_file(topic + '_data_table.csv')
    if answer == 1:
        df = pd.read_csv(topic+'_data_table.csv', index_col=0)
        df = df.drop(['level_0'], axis=1, errors='ignore')#Drops level_0 column that keeps showing up 
    else:
#         df = pd.DataFrame(columns=('file_name'))
        df = pd.DataFrame(columns=('root', 'file', 'file_path', 'file_type', 'raw_text', 'clean_text', \
                                   'word_token', 'key_words','urls', 'emails'))
        df.to_csv(topic + '_data_table.csv')
    return df, answer, topic

#-----------------------------------------------------


df, answer, topic = initialize_data_log() #determine if a data table exists already

#Does the table exist?
if answer == 0:
    print(topic+'_data_table does not exist, so a new one was created.')#Print this if it doesn't
    
    
else:
    print(topic+'_data_table exists, so it is being read.')#print this if it does
    
'''
If data_table exists, compare its file names to the directory's file names.

If data_table does not exist, establish the file and populate it. 

These actions are done in the initialize_data_log function. 

'''

directory = topic + '_data' #name the directory we want to search

file_list = []
table_file_list = list(df['file_path'])
#--------------------------------------------
#Establish a list of the files within a directory

iterator = len(df)            

for file in tqdm(file_list):
    if file in table_file_list: #compare the directory file names to the data table file name list.
        pass                    #If the directory file name is in the data table file name list, pass.
    
    elif file.startswith('._') or file.startswith('.'): # ignore the file names that start with '._' or '.'
        pass                    #If the file starts with either condition, pass.
    
    else: #Set up the new row for further processing
        print('New File:', file)
        df.loc[iterator, 'file_path'] = file # add the pathway name and file name to the df
        accessed, created, modified = accessed_created_modified(file)
        df.loc[iterator, 'last_accessed'] = accessed
        df.loc[iterator, 'created'] = created
        df.loc[iterator, 'last_modified'] = modified

        iterator += 1 # increase the iterator by one so it will load the next at the bottom of the df

file_list = []
table_file_list = list(df['file_path'])

#--------------------------------------------
#Establish a list of the files within a directory
iterator = 0

for root, dirs, files in tqdm(os.walk(directory)):
    for file in files:
        if file.startswith('.'):
            pass
        elif file.endswith('.csv'):
            pass
        else:
            df.loc[iterator, 'root'] = root
            df.loc[iterator, 'file'] = file
#             df.loc[iterator, 'file_name'] = root +'/' + file
#             print('len(files):', len(files))
#             print('root:', root)
#             print('dirs:', dirs)
#             print('files:', files)
            pathway = os.path.join(root, file)
            df.loc[iterator, 'file_path'] = pathway
            pathway = os.path.realpath(pathway)# added from Jeanna for Windows to read
            file_list.append(pathway)
            iterator += 1
            
print(len(file_list))           
if table_file_list == file_list:
    print('There is no change in files.')
# elif len(table_file_list) == 0:
#     print('The data_table is empty.')
elif len(table_file_list) > len(file_list):
    print('A file was deleted.')
elif len(table_file_list) < len(file_list):
    print(str(len(file_list))+' files were added.')
else:
    print('There is a change to the files.')
    
print(len(df))

What topic are we looking at today?human trafficking


0it [00:00, ?it/s]
1it [00:00, 55.41it/s]

human_trafficking_data_table does not exist, so a new one was created.
11
11 files were added.
11


In [2]:
df

,root,file,file_path,file_type,raw_text,clean_text,word_token,key_words,urls,emails
0,human_trafficking_data,Thehardtruthabouthumantrafficking.pdf,human_trafficking_data/Thehardtruthabouthumant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,human_trafficking_data,index.pdf,human_trafficking_data/index.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,human_trafficking_data,An_Introduction_to_Human_Trafficking_-_Backgro...,human_trafficking_data/An_Introduction_to_Huma...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,human_trafficking_data,bc-infosheet-human-trafficking-101-english.pdf,human_trafficking_data/bc-infosheet-human-traf...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,human_trafficking_data,WHO_RHR_12.42_eng.pdf,human_trafficking_data/WHO_RHR_12.42_eng.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,human_trafficking_data,FS36_en.pdf,human_trafficking_data/FS36_en.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,human_trafficking_data,Crime Victim Rights PDF.pdf,human_trafficking_data/Crime Victim Rights PDF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,human_trafficking_data,HumanTrafficking Report.pdf,human_trafficking_data/HumanTrafficking Report...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,human_trafficking_data,2020-TIP-Report-Complete-062420-FINAL.pdf,human_trafficking_data/2020-TIP-Report-Complet...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,human_trafficking_data,fact_sheet_human_trafficking_english.pdf,human_trafficking_data/fact_sheet_human_traffi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Compare file names in data_table with file list

In [3]:
#---------------------------------------------
''' Compare the files within a directory with file names already listed in our csv. 
This is to find files to add to our data table. After finding new files, set up the 
new row with that new file's name for further processing. 

This needs another filter. At the moment it is looking at EVERY file regardless of type and placing it into the DF. 
What we want is it to filter out only the files which we can read and to place those files into a list and 
into the DF. Iterating over my OneDrive file, it is reading 1,043,405 files, but the majority of these files are 
NOT pdfs, docxs, pptxs, txts or rtfs. 


From the EXTRACT RAW TEXT script below, move the subroutine that identifies the file type up to this block. 
'''

if answer == 1:
    iterator = len(df)
else:
    iterator = 0

for file in tqdm(file_list):
    if file in table_file_list: #compare the directory file names to the data table file name list.
        pass                    #If the directory file name is in the data table file name list, pass.
    
    elif file.startswith('._') or file.startswith('.'): # ignore the file names that start with '._' or '.'
        pass                    #If the file starts with either condition, pass.
    
    else: #Set up the new row for further processing
#         print('New File:', file)
        df.loc[iterator, 'file_path'] = file # add the pathway name and file name to the df
        accessed, created, modified = accessed_created_modified(file)
        df.loc[iterator, 'last_accessed'] = accessed
        df.loc[iterator, 'created'] = created
        df.loc[iterator, 'last_modified'] = modified

        iterator += 1 # increase the iterator by one so it will load the next at the bottom of the df

100%|██████████| 11/11 [00:00<00:00, 850.55it/s]


In [4]:
print(df.shape)
display(df)

(11, 13)


,root,file,file_path,file_type,raw_text,clean_text,word_token,key_words,urls,emails,last_accessed,created,last_modified
0,human_trafficking_data,Thehardtruthabouthumantrafficking.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:24 2020
1,human_trafficking_data,index.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:58 2020
2,human_trafficking_data,An_Introduction_to_Human_Trafficking_-_Backgro...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:02 2020
3,human_trafficking_data,bc-infosheet-human-trafficking-101-english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:51 2020
4,human_trafficking_data,WHO_RHR_12.42_eng.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:16 2020
5,human_trafficking_data,FS36_en.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:44:07 2020
6,human_trafficking_data,Crime Victim Rights PDF.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:52:17 2020
7,human_trafficking_data,HumanTrafficking Report.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:41 2020
8,human_trafficking_data,2020-TIP-Report-Complete-062420-FINAL.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:46:32 2020
9,human_trafficking_data,fact_sheet_human_trafficking_english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:22 2020


## Find deleted files since our last run

In [5]:
#-----------------------------------------        
#Find deleted directory files that are listed in the data table, but are not in the directory

print("\n\nI am checking for deleted files")

for file in tqdm(table_file_list):
    try:
        if file in file_list or file.endswith('.csv'): #If the file exists in the directory and the data table, continue.
            continue 
        else:              #If the file no longer exists within the directory but is in the data table,
                           #we want to delete the entry in the data table and reset the index.
            print('_'*72) #Cosmetic line to separate identified file
            print(file, 'no longer exists!')
            try:
                df = df[~df.file_name.str.contains(file)]# the new df drops file 
                df = df.reset_index(drop=True)
                print(file, 'was dropped.')
            except KeyError:
                print('KeyError met')
                print('_'*72) #Cosmetic line to separate identified file
                continue
            except ValueError:
                print('ValueError met with file:', df.index[df['file_name'] == file])
                print('_'*72) #Cosmetic line to separate identified file
                continue
    except AttributeError:
        print('AttributeError met with', file)
        continue
# df.to_csv('data_table.csv')

0it [00:00, ?it/s]



I am checking for deleted files


In [6]:
print(df.shape)
display(df)

(11, 13)


,root,file,file_path,file_type,raw_text,clean_text,word_token,key_words,urls,emails,last_accessed,created,last_modified
0,human_trafficking_data,Thehardtruthabouthumantrafficking.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:24 2020
1,human_trafficking_data,index.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:58 2020
2,human_trafficking_data,An_Introduction_to_Human_Trafficking_-_Backgro...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:02 2020
3,human_trafficking_data,bc-infosheet-human-trafficking-101-english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:51 2020
4,human_trafficking_data,WHO_RHR_12.42_eng.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:16 2020
5,human_trafficking_data,FS36_en.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:44:07 2020
6,human_trafficking_data,Crime Victim Rights PDF.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:52:17 2020
7,human_trafficking_data,HumanTrafficking Report.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:41 2020
8,human_trafficking_data,2020-TIP-Report-Complete-062420-FINAL.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:46:32 2020
9,human_trafficking_data,fact_sheet_human_trafficking_english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:22 2020


## Extract raw text, clean it, do word count, tokenize words, then place it in the data_table

In [7]:
def pdf_to_txt(path):
    #from pdfminer.six documentation: https://pdfminersix.readthedocs.io/en/latest/tutorial/composable.html
    from io import StringIO
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfdocument import PDFDocument
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.pdfparser import PDFParser
    output_string = StringIO()
    with open(path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    text = str(output_string.getvalue())
    return text

In [8]:
runtime = round((len(df['file']) * 2.14) / 60, 2)

print("This will take about", runtime, "minutes for me to read all", len(df['file']), 'files. ')

This will take about 0.39 minutes for me to read all 11 files. 


In [9]:
df

,root,file,file_path,file_type,raw_text,clean_text,word_token,key_words,urls,emails,last_accessed,created,last_modified
0,human_trafficking_data,Thehardtruthabouthumantrafficking.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:24 2020
1,human_trafficking_data,index.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:58 2020
2,human_trafficking_data,An_Introduction_to_Human_Trafficking_-_Backgro...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:02 2020
3,human_trafficking_data,bc-infosheet-human-trafficking-101-english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:51 2020
4,human_trafficking_data,WHO_RHR_12.42_eng.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:16 2020
5,human_trafficking_data,FS36_en.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:44:07 2020
6,human_trafficking_data,Crime Victim Rights PDF.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:52:17 2020
7,human_trafficking_data,HumanTrafficking Report.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:41 2020
8,human_trafficking_data,2020-TIP-Report-Complete-062420-FINAL.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:46:32 2020
9,human_trafficking_data,fact_sheet_human_trafficking_english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:22 2020


In [10]:
from gensim.utils import lemmatize
import spacy
import numpy as np

#------------------------------------------
#Extract raw text and clean it, then place it in data table

print("\nI'm extracting, cleaning, and reading each file right now. \n")

#-----------------------------------------
# Cleaning raw text through dataframe outside the main loop
for iterator in tqdm(range(len(df))):
    if pd.isnull(df.loc[iterator, 'raw_text']):
        try:
            if file.endswith('.pdf'):
                df.loc[iterator, 'file_type'] = 'pdf'
            elif file.endswith('.txt'):
                df.loc[iterator, 'file_type'] = 'txt'
            elif file.endswith('.doc'):
                df.loc[iterator, 'file_type'] = 'doc'
            elif file.endswith('.docx'):
                df.loc[iterator, 'file_type'] = 'docx'
            elif file.endswith('.rtf'):
                df.loc[iterator, 'file_type'] = 'rtf'
            elif file.endswith('.pptx'):
                df.loc[iterator, 'file_type'] = 'pptx'
#             elif file.endswith('.jar'):
#                 continue
            else:
                df.loc[iterator, 'file_type'] = 'Not Readable'

        except StopIteration:
            df.loc[iterator, 'error_status'] = 'StopIteration'
            continue
            
        except RuntimeError:
            df.loc[iterator, 'error_status'] = 'RuntimeError'
            continue
        
        except ValueError:
            df.loc[iterator, 'error_status'] = 'ValueError'
            continue
        
        except UnicodeDecodeError:
            df.loc[iterator, 'error_status'] = 'UnicodeDecodeError'
            continue
    
        #Extract raw text
        file_types_readable = ['pdf', 'txt', 'doc', 'docx', 'rtf', 'pptx']
        
        if df.loc[iterator, 'file_type'] not in file_types_readable:
            pass
        else:
            try:
                #Save raw text to DataFrame
                df.loc[iterator, 'raw_text'] = str(textract.process(str(df.loc[iterator, 'file_path']), encode='ascii'))
                
            except UnicodeDecodeError:
                df.loc[iterator, 'error_status'] = 'UnicodeDecodeError'
                continue
                
#             except UnicodeDecodeError:
#                 df.loc[iterator, 'raw_text'] = str(textract.process(df.loc[iterator, 'file_name'], \
#                                             encoding='latin-1'))#Save raw text to DataFrame
#             except TypeError:
#                 print('TypeError on file', iterator)
#                 continue

#             except ShellError:
#                 print('Line 66: ShellError on file', df.loc[iterator, 'file_name'])
#                 continue

            except NameError:
                df.loc[iterator, 'error_status'] = 'NameError'
                continue

            except ModuleNotFoundError:
                df.loc[iterator, 'error_status'] = 'ModuleNotFoundError'
                continue
                
            except:
                df.loc[iterator, 'error_status'] = 'Unspecified_Error'
                continue
            
        
    
        #Clean raw text
        x = str(clean_the_text(df.loc[iterator, 'raw_text'], remove_numbers=False))
        x = x.replace(' x93 ', ': ')
        x = x.replace('...........................................................................................', '')
        x = x.replace(' s ', "'s ")
        x = x.replace(' xac x81', ' fi')
        x = x.replace('xc3 x9f', 'copyright')
        x = x.replace(' et al ', 'etal')
        x = x.replace('x97 x8f ', '*')
        x = x.replace(' x937 ', ': ')
        x = x.replace(' xac x82', 'sp')
        df.loc[iterator, 'clean_text'] = x

        #Tokenize clean text
        df.loc[iterator, 'word_token'] = str(tokenize_by_words2(str(df.loc[iterator, 'clean_text'])))

        #Word count clean text
        df.loc[iterator, 'word_count'] = int(count_words(df.loc[iterator, 'clean_text']))

        #Pull email addresses
        df.loc[iterator, 'emails'] = str(re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', str(df.loc[iterator, 'clean_text'])))

        #Find keywords using Gensim
        try:
            df.loc[iterator, 'key_words'] = keywords(str(df.loc[iterator, 'clean_text']))
            df.loc[iterator, 'key_words'] = df.loc[iterator, 'key_words'].replace('\n', ', ')
        except RuntimeError:
            pass


  0%|          | 0/11 [00:00<?, ?it/s]


I'm extracting, cleaning, and reading each file right now. 



100%|██████████| 11/11 [00:57<00:00,  5.25s/it]


In [11]:
df.tail(4)

,root,file,file_path,file_type,raw_text,clean_text,word_token,key_words,urls,emails,last_accessed,created,last_modified,word_count
7,human_trafficking_data,HumanTrafficking Report.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'WHAT IS HUMAN TRAFFICKING?\n\nEXPLORING THE ...,bWHAT IS HUMAN TRAFFICKING? EXPLORING THE SCOP...,"['bWHAT', 'IS', 'HUMAN', 'TRAFFICKING?', 'EXPL...","trafficking, traffickers, trafficked, traffick...",NaN,[],"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:41 2020,17687.0
8,human_trafficking_data,2020-TIP-Report-Complete-062420-FINAL.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'T\nR\nA\nF\nF\nI\nC\nK\nI\nNGI\nNP\nE\nR\nS\...,bT R A F F I C K I NGI NP E R S ONSR E P OR T ...,"['bT', 'R', 'A', 'F', 'F', 'I', 'C', 'K', 'I',...","trafficking, traffickers, trafficker, traffick...",NaN,['tipreport@state.gov'],"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:46:32 2020,570934.0
9,human_trafficking_data,fact_sheet_human_trafficking_english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'HUMAN TRAFFICKING\n\nHuman trafficking is a ...,bHUMAN TRAFFICKING Human trafficking is a form...,"['bHUMAN', 'TRAFFICKING', 'Human', 'traffickin...","trafficking, trafficked, traffickers, traffick...",NaN,[],"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:22 2020,808.0
10,human_trafficking_data,211980.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,"b""The author(s) shown below used Federal funds...",The author(s) shown below used Federal funds p...,"['The', 'author(s)', 'shown', 'below', 'used',...","trafficking, traffickers, trafficked, traffick...",NaN,['children@interpol.int'],"Nov 18, 2020, 06:07",Wed Nov 18 05:15:04 2020,Wed Nov 18 04:46:52 2020,79486.0


In [12]:
df.isnull().sum()

root              0
file              0
file_path         0
file_type         0
raw_text          0
clean_text        0
word_token        0
key_words         0
urls             11
emails            0
last_accessed     0
created           0
last_modified     0
word_count        0
dtype: int64

In [13]:
%%time

df.to_csv(topic + '_data_table.csv')

CPU times: user 496 ms, sys: 64.2 ms, total: 560 ms
Wall time: 563 ms


In [14]:
df.shape

(11, 14)

In [15]:
df.T

,0,1,2,3,4,5,6,7,8,9,10
root,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data
file,Thehardtruthabouthumantrafficking.pdf,index.pdf,An_Introduction_to_Human_Trafficking_-_Backgro...,bc-infosheet-human-trafficking-101-english.pdf,WHO_RHR_12.42_eng.pdf,FS36_en.pdf,Crime Victim Rights PDF.pdf,HumanTrafficking Report.pdf,2020-TIP-Report-Complete-062420-FINAL.pdf,fact_sheet_human_trafficking_english.pdf,211980.pdf
file_path,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...
file_type,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf
raw_text,"b'See discussions, stats, and author profiles ...",b'U.S. Department of Health and Human Services...,"b'Vienna International Centre, PO Box 500, 140...",b'Human Trafficking 101\nWhat is Human Traffic...,b'Understanding and addressing\nviolence again...,b'Human Rights\nand Human Trafficking\n\nFact ...,"b""CRIME VICTIM RIGHTS\nState and federal laws ...",b'WHAT IS HUMAN TRAFFICKING?\n\nEXPLORING THE ...,b'T\nR\nA\nF\nF\nI\nC\nK\nI\nNGI\nNP\nE\nR\nS\...,b'HUMAN TRAFFICKING\n\nHuman trafficking is a ...,"b""The author(s) shown below used Federal funds..."
clean_text,"bSee discussions, stats, and author profiles f...","bU.S. Department of Health and Human Services,...","bVienna International Centre, PO Box 500, 1400...",bHuman Trafficking 101 What is Human Trafficki...,bUnderstanding and addressing violence against...,bHuman Rights and Human Trafficking Fact Sheet...,CRIME VICTIM RIGHTS State and federal laws acc...,bWHAT IS HUMAN TRAFFICKING? EXPLORING THE SCOP...,bT R A F F I C K I NGI NP E R S ONSR E P OR T ...,bHUMAN TRAFFICKING Human trafficking is a form...,The author(s) shown below used Federal funds p...
word_token,"['bSee', 'discussions,', 'stats,', 'and', 'aut...","['bU.S.', 'Department', 'of', 'Health', 'and',...","['bVienna', 'International', 'Centre,', 'PO', ...","['bHuman', 'Trafficking', '101', 'What', 'is',...","['bUnderstanding', 'and', 'addressing', 'viole...","['bHuman', 'Rights', 'and', 'Human', 'Traffick...","['CRIME', 'VICTIM', 'RIGHTS', 'State', 'and', ...","['bWHAT', 'IS', 'HUMAN', 'TRAFFICKING?', 'EXPL...","['bT', 'R', 'A', 'F', 'F', 'I', 'C', 'K', 'I',...","['bHUMAN', 'TRAFFICKING', 'Human', 'traffickin...","['The', 'author(s)', 'shown', 'below', 'used',..."
key_words,"victim, victimization, trafficked, traffickers...","services, trafficking, trafficked, traffickers...","trafficking, trafficked, traffickers, traffick...","victims, victim, trafficking, trafficked, traf...","trafficked, traffickers, health, human traffic...","trafficking, traffickers, trafficked persons, ...","victim, victims, rights, right, informed, info...","trafficking, traffickers, trafficked, traffick...","trafficking, traffickers, trafficker, traffick...","trafficking, trafficked, traffickers, traffick...","trafficking, traffickers, trafficked, traffick..."
urls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emails,[],[],['ahtu@unodc.org'],[],[],[],[],[],['tipreport@state.gov'],[],['children@interpol.int']


## Extract Words Based upon Part of Sentence (Part of Speech)

This script is pulling the parts of speech (POS) for each word in each clean_text file. It places it into a list, then makes that list into a set to get rid of duplicates. 

I am not satisfied with its accuracy. It is taking guesses, but needs refinement. 

The purpose of this section is to provide some preliminary coding experimentation along the lines of identifying POS that we can incorporate into the "concept" search later. The Concept Search is only pulling synonyms of the search term at the moment. This is pulling in all synonyms, even those which may be out of context for the concept and the sentence. My belief is that in order to pull concepts, the synonyms must equal the part of speech that the search_term occupies. Also, I believe that the Concept Search must take into account the words around the synonym. For example, when I do a concept search for the word 'violence', the concept search is not taking into account positive and negative adjectives associated with the synonyms of 'violence'. This is a refinement area. 

In [16]:
for iterator in tqdm(range(len(df)-1)):
    a = df.loc[iterator, 'clean_text']
    features = tokenize_by_words(str(a))

    tags = pos_tag(str(a))

    proper_names = []
    nouns = []
    verbs = []
    adjectives = []
    adverbs = []
    possessives = []
    pronouns = []

    for tag in tags:
        if "NNP" in tag or 'NNPS' in tag:
            proper_names.append(tag[0].lower())
        elif 'NN' in tag or 'NNS' in tag:
            nouns.append(tag[0].lower())
        elif "VB" in tag or 'VBD' in tag or 'VBG' in tag or 'VBN' in tag or 'VBP' in tag or 'VBZ' in tag:
            verbs.append(tag[0].lower())
        elif 'PRP$' in tag:
            possessives.append(tag[0].lower())
        elif 'JJ' in tag or 'JJR' in tag or 'JJS' in tag:
            adjectives.append(tag[0].lower())
        elif 'RB' in tag or 'RBR' in tag or 'RBS' in tag:
            adverbs.append(tag[0].lower())
        elif 'PRP' in tag or 'PRP$' in tag:
            pronouns.append(tag[0].lower())

    
    if len(proper_names) > 0:
        df.loc[iterator,'proper_names'] = str(set(proper_names))
    if len(nouns) >0:
        df.loc[iterator,'nouns'] = str(set(nouns))
    if len(verbs) >0:
        df.loc[iterator,'verbs'] = str(set(verbs))
    if len(adjectives) > 0:
        df.loc[iterator,'adjectives'] = str(set(adjectives))
    if len(adverbs) > 0:
        df.loc[iterator,'adverbs'] = str(set(adverbs))
    if len(possessives) > 0:
        df.loc[iterator,'possessives'] = str(set(possessives))
    if len(pronouns) > 0:
        df.loc[iterator,'pronouns'] = str(set(pronouns))



100%|██████████| 10/10 [00:35<00:00,  3.58s/it]


In [17]:
df.T

,0,1,2,3,4,5,6,7,8,9,10
root,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data,human_trafficking_data
file,Thehardtruthabouthumantrafficking.pdf,index.pdf,An_Introduction_to_Human_Trafficking_-_Backgro...,bc-infosheet-human-trafficking-101-english.pdf,WHO_RHR_12.42_eng.pdf,FS36_en.pdf,Crime Victim Rights PDF.pdf,HumanTrafficking Report.pdf,2020-TIP-Report-Complete-062420-FINAL.pdf,fact_sheet_human_trafficking_english.pdf,211980.pdf
file_path,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,/Users/benmeredith/Desktop/GitHub/Wesley/human...
file_type,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf,pdf
raw_text,"b'See discussions, stats, and author profiles ...",b'U.S. Department of Health and Human Services...,"b'Vienna International Centre, PO Box 500, 140...",b'Human Trafficking 101\nWhat is Human Traffic...,b'Understanding and addressing\nviolence again...,b'Human Rights\nand Human Trafficking\n\nFact ...,"b""CRIME VICTIM RIGHTS\nState and federal laws ...",b'WHAT IS HUMAN TRAFFICKING?\n\nEXPLORING THE ...,b'T\nR\nA\nF\nF\nI\nC\nK\nI\nNGI\nNP\nE\nR\nS\...,b'HUMAN TRAFFICKING\n\nHuman trafficking is a ...,"b""The author(s) shown below used Federal funds..."
clean_text,"bSee discussions, stats, and author profiles f...","bU.S. Department of Health and Human Services,...","bVienna International Centre, PO Box 500, 1400...",bHuman Trafficking 101 What is Human Trafficki...,bUnderstanding and addressing violence against...,bHuman Rights and Human Trafficking Fact Sheet...,CRIME VICTIM RIGHTS State and federal laws acc...,bWHAT IS HUMAN TRAFFICKING? EXPLORING THE SCOP...,bT R A F F I C K I NGI NP E R S ONSR E P OR T ...,bHUMAN TRAFFICKING Human trafficking is a form...,The author(s) shown below used Federal funds p...
word_token,"['bSee', 'discussions,', 'stats,', 'and', 'aut...","['bU.S.', 'Department', 'of', 'Health', 'and',...","['bVienna', 'International', 'Centre,', 'PO', ...","['bHuman', 'Trafficking', '101', 'What', 'is',...","['bUnderstanding', 'and', 'addressing', 'viole...","['bHuman', 'Rights', 'and', 'Human', 'Traffick...","['CRIME', 'VICTIM', 'RIGHTS', 'State', 'and', ...","['bWHAT', 'IS', 'HUMAN', 'TRAFFICKING?', 'EXPL...","['bT', 'R', 'A', 'F', 'F', 'I', 'C', 'K', 'I',...","['bHUMAN', 'TRAFFICKING', 'Human', 'traffickin...","['The', 'author(s)', 'shown', 'below', 'used',..."
key_words,"victim, victimization, trafficked, traffickers...","services, trafficking, trafficked, traffickers...","trafficking, trafficked, traffickers, traffick...","victims, victim, trafficking, trafficked, traf...","trafficked, traffickers, health, human traffic...","trafficking, traffickers, trafficked persons, ...","victim, victims, rights, right, informed, info...","trafficking, traffickers, trafficked, traffick...","trafficking, traffickers, trafficker, traffick...","trafficking, trafficked, traffickers, traffick...","trafficking, traffickers, trafficked, traffick..."
urls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emails,[],[],['ahtu@unodc.org'],[],[],[],[],[],['tipreport@state.gov'],[],['children@interpol.int']


In [18]:
df.to_csv(topic + '_data_table.csv')

## Find duplicate content but different file name

In [19]:
%%time
#-------------------------------------------
#Find duplicate content but different file name and report which is the newer file
''' Still need to return the name of the newest file.Want to also give the user the choice 
    of moving one of the files to a new directory or deleting the additional file. '''

#import os

for iterator_1 in tqdm(range(len(df))):
    for iterator_2 in range(len(df)):
        try:
            if df.loc[iterator_1, 'raw_text'] == df.loc[iterator_2, 'raw_text'] and \
                df.loc[iterator_1, 'file_path'] == df.loc[iterator_2, 'file_path'] and \
                iterator_1 != iterator_2:
               
                print('Suspected duplicate: \n\t', df.loc[iterator_1, 'file_name'], '& \n\t', \
                      df.loc[iterator_2, 'file_name'])
                print(df.loc[iterator_1, 'file_name'],'created', df.loc[iterator_1, 'created'], \
                     '\n', df.loc[iterator_2, 'file_name'], 'created', df.loc[iterator_2, 'created'])
                
                print('_'*72)
                
                
        except KeyError:
            continue
            
        except FileNotFoundError:
            continue


100%|██████████| 11/11 [00:00<00:00, 4274.75it/s]

CPU times: user 4.66 ms, sys: 1.65 ms, total: 6.32 ms
Wall time: 5.03 ms


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   root           11 non-null     object 
 1   file           11 non-null     object 
 2   file_path      11 non-null     object 
 3   file_type      11 non-null     object 
 4   raw_text       11 non-null     object 
 5   clean_text     11 non-null     object 
 6   word_token     11 non-null     object 
 7   key_words      11 non-null     object 
 8   urls           0 non-null      object 
 9   emails         11 non-null     object 
 10  last_accessed  11 non-null     object 
 11  created        11 non-null     object 
 12  last_modified  11 non-null     object 
 13  word_count     11 non-null     float64
 14  proper_names   10 non-null     object 
 15  nouns          10 non-null     object 
 16  verbs          10 non-null     object 
 17  adjectives     10 non-null     object 
 18  adverbs     

# Move files to new directory

This is just a test on how to do it. We will need to refine this for our needs. 

In [ ]:
def pdf_to_txt(path):
    # Load libraries -----------------------------------------------------
    import pdfminer
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO

    try:
        # ---------- Set parameters ---------------
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, laparams=laparams)
        with open(path, 'rb') as fp:
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            password = ""
            maxpages = 0
            caching = True
            pagenos=set()

            # --------Actual text pull -------------------------
            for page in PDFPage.get_pages(fp, 
                                          pagenos, 
                                          maxpages=maxpages, 
                                          password=password,
                                          caching=caching, 
                                          check_extractable=True,
                                         ):
                interpreter.process_page(page)

            text = retstr.getvalue()

            #-------Close files after reading them -----------------
            device.close()
            retstr.close()
            return text

    except UnicodeDecodeError:

        text = 'ascii error'
        return text
    
    
def docx_to_txt(path):
    WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
    PARA = WORD_NAMESPACE + 'p'
    TEXT = WORD_NAMESPACE + 't'


    try:
        from xml.etree.cElementTree import XML
    except ImportError:
        from xml.etree.ElementTree import XML
    import zipfile
    """
    Take the path of a docx file as argument, return the text in unicode.
    """
    with zipfile.ZipFile(path) as document:
        xml_content = document.read('word/document.xml')
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.getiterator(PARA):
        texts = [node.text
                 for node in paragraph.getiterator(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))

    return '\n\n'.join(paragraphs)

def txt_to_txt(path):
    try:
        with open(path, 'rt', errors='ignore') as tempopen: # open the file for reading text
            temptext = tempopen.read()  # read the entire file as a string
            
    except TypeError:
        with open(path, 'rt', encode='utf-8') as tempopen:# open the file for reading text
            temptext = tempopen.read()  # read the entire file as a string
            
    return temptext                 # return the raw text


def error_file_search(verbose = True):
    
    import os
    import re
    
    input_directory_name = '/Users/benmeredith/Desktop/Wesley_mock/data/school_violence' # name of the directory that contains input files
    error_directory_name = 'error_dir' # name of the directory for unreadable files
    error_file_list = []
    
    # create new directory to put error files if it hasn't been made before
    if not os.path.exists(error_directory_name):
        os.makedirs(error_directory_name)

    # list of files in the input directory
    filenames = os.listdir(input_directory_name)

    # dictionary of functions
    func_dict = {'pdf':pdf_to_txt, 'doc':docx_to_txt, 'docx':docx_to_txt, 'txt':txt_to_txt}

    # function to get file type (pdf, doc, docx, txt, jpg, etc)
    def get_type(filename):
        pattern = re.compile("\.([A-Za-z]+)$")
        try:
            file_type = pattern.search(filename).group(1)
        except:
            file_type = ""
        finally:
            return file_type


    for filename in filenames:
    
        if verbose: print(f"..........\nWorking on {filename}\n..........")
    
        file_type = get_type(filename)
        file_location = os.path.join(input_directory_name, filename)
        file_destination = os.path.join(error_directory_name, filename)

        # check if the file doesn't have type and it's not on the func_dict.keys()
        if file_type == "" or file_type not in func_dict.keys():
            
            if verbose: print(f"\nError found!\nMoving to {error_directory_name}...\n\n")
                
            # move file to the error directory
            os.replace(file_location, file_destination)
            error_file_list.append(filename)
            
            continue
            
        # grab function from the dictionary
        func = func_dict[file_type]

        try:
            res = func(file_location)
        except:
            if verbose: print(f"\nError found!\nMoving to {error_directory_name}...\n\n")
                
            # move file to the error directory
            os.replace(file_location, file_destination)
            error_file_list.append(filename)
            
        else:
            if res == "ascii error":
                if verbose: print(f"\nError found!\nMoving to {error_directory_name}...\n\n")
                
                # move file to the error directory
                os.replace(file_location, file_destination)
                error_file_list.append(filename)
            elif verbose:
                print(f"\nNo error found on this file\n\n")
            else:
                continue
    
    
    # printing report
    print("\n\n\n................\nDone!\n................\n")
    print("Error files: ")
    for error_file in error_file_list:
        print(error_file)
        


In [ ]:
error_file_search()

In [ ]:
df.tail()

# Fill Out syn_table.csv

This section of code takes the synonyms for each word in the syn_table, finds the synonyms for each synonym, and saves it to the dataframe. 

In [ ]:
# Load libraries
import pandas as pd
%run lib/__wesley_init__.py

# Load csv into dataframe
syndf = pd.read_csv('syn_table.csv', index_col = 0)

full_list_of_words = []
synonyms = list(syndf['synonyms'])


for list_of_words in synonyms:
    words = str(list_of_words)
    for word in words.split(', '):
        full_list_of_words.append(word)

full_list_of_words = set(full_list_of_words)
full_list_of_words.remove('nan')
full_list_of_words.remove('')
    
for word in full_list_of_words:
    print(word)
    if word == 'nan':
        print('@'*72)
        pass
        
    else:
        try: 
            print('Working on word', word)
            if word in list(syndf['word']):
                print(word, 'is in synonym table.')
                continue

            elif word not in list(syndf['word']):
                print(word, 'is not in synonym table.')

                counter = len(syndf)+1

                synonyms = get_synonyms(word)
                synonyms = str(synonyms)
                synonyms = synonyms.replace('[', '')
                synonyms = synonyms.replace("'", '')
                synonyms = synonyms.replace(']', '')

                syndf.loc[counter, 'word'] = word
                syndf.loc[counter, 'synonyms'] = synonyms

        except AttributeError:
            pass

        except:
            pass

syndf.to_csv('syn_table.csv')

In [ ]:
syndf.tail()

In [ ]:
syndf.to_csv('syn_table.csv')

# Initialize syn_table.csv

In [ ]:
import pandas as pd
%run lib/__wesley_init__.py

syndf, synanswer = initialize_syn_log()

if synanswer == 0:
    print('I could not find the concept data table, so I established one.')
else:
    print('I have loaded the concept data table for searches.')

# First Search Code

In [ ]:
#Load the libraries and data_table
import pandas as pd
from nltk import tokenize
from IPython.display import display, HTML
import platform
import numpy as np
%run lib/__wesley_init__.py

df = pd.read_csv('data_table.csv', index_col=0)

#---------------------------------
#Search
runcontinue = 'yes'

while runcontinue != 'quit':
    search_type = input("Would you like me to search 'content' or just provide the 'titles' where the word appears?")
    
    search_term = input("\nSearch for: ")
    search_term = str(search_term).lower()
        
    search_term = ' ' + search_term + ' '
    search_term = search_term.lower()
    
    if search_type == 'content':
        for iterator, article in enumerate(df['clean_text']):
            article = str(article).split('. ')#Simple split - needs to be for all punctuation
            for sentence in article:
                if search_term in sentence:
                    print('-'*72)
                    print('['+df.loc[iterator, 'file_name']+']', '\n')
                    
                    #create string template for HTML link
                    link = "<a href='{href}'> Link to article </a>"
                    system = platform.system()
                    if system == 'Darwin':
                        result_file = 'File://'+ df.loc[iterator, 'file_name']
                    elif system == 'Windows':
                        result_file = 'C:'+ df.loc[iterator, 'file_name']
                    
                    # create HTML object, using the string template
                    html = HTML(link.format(href=result_file))
                    
                    # display the HTML object to put the link on the page:
                    display(html)
                    
                    '''Next line highlights the term'''
                    print((sentence.replace(search_term, '\033[43;30m{}\033[m'.format(search_term))), '\n\n')

#             try:
#                 article = str(article).split('. ')#Simple split - needs to be for all punctuation
#                 for sentence in article:
#                     if search_term in sentence:
#                         print('-'*72)
#                         print('[['+df.loc[iterator, 'file_name']+']]', '\n\n')
#                         '''Next line highlights the term'''
#                         print((sentence.replace(search_term, '\033[43;30m{}\033[m'.format(search_term))), '\n\n')
                        
#             except TypeError:
#                 continue

    elif search_type == 'titles' or search_type == 'Titles':   
        for iterator, article in enumerate(df['file_name']):
            try:
                if search_term in article:
                    print('\n'+ search_term + ' is in ' + df.loc[iterator, 'file_name'])
            except TypeError:
                continue

            
#-------------------------------------------------------
    # Would you like to do a synonym search of the term?
    ''' This is not pulling synonyms, but when it does it disregards part of speech. 
        For example, a synonym for 'bank' is 'trust'. When it pulls this synonym, it is 
        pulling the verb 'trust' and not the noun 'trust'. But this will open up other
        problems as 'trust' as a noun can be either a monetary instrument or the 
        result of trusting someone. This needs refinement. 
    '''
    print('*'*72, '\n', '*'*72, '\n\n')
    synonym_search = input('\n\nWould you like to do a concept search? (yes or no) ')
    
    syndf = pd.read_csv('syn_table.csv', index_col=0)
    
    synonym_search = synonym_search.lower()
        
    if synonym_search == 'yes':
              
        search_term = search_term.strip()
        
        iterator = 1
        
        if search_term in str(syndf['word']):
            print('The search term is in the dataset.')
            
            for word in syndf['word']:
                word = word.strip()
    
                if word == search_term:
                    print(word, search_term)
                    synonyms = syndf.loc[iterator, 'synonyms']
                    if synonyms != np.nan:
                        continue
                    else: iterator +=1
                else:
                    iterator += 1

        else: 
            counter = len(syndf)+1
            
            syndf.loc[counter, 'word'] = search_term
            synonyms = get_synonyms(search_term)
            synonyms = str(synonyms)
            synonyms = synonyms.replace('[', '')
            synonyms = synonyms.replace("'", '')
            synonyms = synonyms.replace(']', '')
            
            syndf.loc[counter, 'synonyms'] = synonyms

            syndf.to_csv('syn_table.csv')


        print('Concepts for ' + search_term + ' are', synonyms)    
        
        search_type = search_type.lower()
        
        if search_type == 'content':
            for iterator, article in enumerate(df['clean_text']):
                try:
                    ''' tokenizing by sentences is not working cleanly. It is not 
                    splitting along punctuation marks. This is the case with the 
                    NLTK library function below AND with our own split_by_sentences() function. '''
                    
                    article = tokenize.sent_tokenize(str(article))#split the text into sentences
                    
                    for sentence in article:
                        if synonyms == np.nan:
                            print("synonyms are not nan")
                            for syn in synonyms.split(', '):
                                syn = ' ' + syn + ' '
                                if syn in sentence:
                                    print('-'*72)
                                    print('[['+df.loc[iterator, 'file_name']+']]', '\n')

                                    #create string template for HTML link
                                    link = "<a href='{href}'> Link to article </a>"
                                    system = platform.system()
                                    if system == 'Darwin':
                                        result_file = 'File://'+ df['file_name'][iterator]
                                    elif system == 'Windows':
                                        result_file = 'C:'+ df['file_name'][iterator]

                                    # create HTML object, using the string template
                                    html = HTML(link.format(href=result_file))

                                    # display the HTML object to put the link on the page:
                                    display(html)

                                    #Next line highlights the term
                                    print((sentence.replace(syn, '\033[43;30m{}\033[m'.format(syn))), '\n\n')
                        else:
                            break
                except TypeError:
                    continue


        elif search_type == 'titles':   
            for iterator, title in enumerate(df['file_name']):
                for syn in synonyms.split(', '):
                    syn = ' ' + syn + ' '
                    try:
                        if syn in title:
                            print('\n'+ syn + ' is in ' + df.loc[iterator, 'file_name'])
                            print('-'*72)
                            print('[['+df.loc[iterator, 'file_name']+']]', '\n')

                            #create string template for HTML link
                            link = "<a href='{href}'> Link to article </a>"
                            system = platform.system()
                            if system == 'Darwin':
                                result_file = 'File://'+ df.loc[iterator, 'file_name']
                            elif system == 'Windows':
                                result_file = 'C:'+ df.loc[iterator, 'file_name']

                            # create HTML object, using the string template
                            html = HTML(link.format(href=result_file))

                            # display the HTML object to put the link on the page:
                            display(html)

                            '''Next line highlights the term'''
                            print((sentence.replace(syn, '\033[43;30m{}\033[m'.format(syn))), '\n\n')

                    except TypeError:
                        continue


    runcontinue = input('To quit, type quit. Otherwise, hit ENTER to make another inquiry: ')

In [ ]:
syndf.tail()

# Testing pdfkit

https://pypi.org/project/pdfkit/ & https://github.com/JazzCore/python-pdfkit

***Evaluation***: PDFkit solves the problem of scraping websites, but it takes far too long to operate. It is good to download websites as pdfs, but PDFKit is taking far too long to download even 262 sites from the net. This is not going to be the way for us to ove this forward.

We need to find a better system. 

In [ ]:
# !pip3 install pdfkit

In [ ]:
# !pip install --upgrade pip

In [ ]:
# !pip3 install wkhtmltopdf

In [ ]:
import pdfkit

site = 'https://www.bbc.com/news/business-46507514'
outpdf = 'data/school_violence_0.pdf'

pdfkit.from_url(site, outpdf)

In [ ]:
# An attempt at looping through a dataframe of urls to pull the pages as pdfs. 

#import necessary libraries
import pandas as pd
import pdfkit
from tqdm import tqdm
import urllib.request
import requests

#read in dataframe resulting from Kelvin's URL Pull Function
url_df = pd.read_csv('url_results.csv')

#run through each URL in the table and register its status code
#then save that status code to the dataframe for reference
iterator = 0
for url in tqdm(url_df.url):
    
    '''
        usnews.com takes forever to load - so I am excluding it for now.
        Amazon.com returns book titles which are of no use to us. 
        IMDb.com returns movies. 
        Goodreads returns book comments.
        For NSV, we don't need to download our own website.
    '''
    
    if 'usnews.com' in url or \
        'amazon.com' in url or \
        'imdb.com' in url or \
        'goodreads.com' in url or \
        'noschoolviolence.org' in url: 
        url_df.loc[iterator, 'status_code'] = 'Website exclusion'
        iterator += 1
        continue
    else:
        try:
            request = requests.get(url)
            url_df.loc[iterator, 'status_code'] = str(request.status_code)
            iterator += 1
        except: 
            url_df.loc[iterator, 'status_code'] = 'exception'
            iterator += 1
            continue

'''In the next loop we run through the dataframe from the above loop and download all of the sites
that have a 200 status code as a pdf into the school_violence directory.

Next, we mark the status appropriately in the dataframe.'''

for count, url in tqdm(enumerate(url_df['url'])):
    try:
        if url.endswith('pdf') and url_df.loc[count, 'status_code'] == '200':
            site = url
            split_site = site.split('/')
            file = split_site[-1]
            urllib.request.urlretrieve(url, 'data/school_violence/'+file+'.pdf')
            url_df.loc[count, 'status'] = 'downloaded'
        elif url_df.loc[count, 'status_code'] == '200': 
                site = url
                split_site = site.split('/')
    #             print(split_site)
                count = str(count)
                pdfkit.from_url(site, 'data/school_violence/'+split_site[-1]+'.pdf')
                url_df.loc[count, 'status'] = 'pdf_made'
    except:
        url_df.loc[count, 'status'] = 'problem'
        print('Problem with this url:', url)

url_df.head()

In [ ]:
url_df['status_code'].value_counts()

In [ ]:
url_df.to_csv('url_results.csv')

# Find Duplicates in the URL Pull

After a new Bing scrape is done, we want to pull out duplicate URLs so we are not pulling them a second time. 

Order of tasks:
1. Bing Search Scrape to existing url_results.csv
1. remove the second duplicate url if one exists
1. go to the first new row where no pull has been done and start to pull new sites as PDFs
1. store the new pdfs in the same folder as the others
1. open the data_table.csv
1. go to the first new file 
1. go to the first open row
1. preprocess the pdf into the data_table.csv

In [ ]:
url_df.drop_duplicates(subset='url', keep='first', inplace=True)


# Testing PyMuPDF

[https://pypi.org/project/PyMuPDF/]
[https://pymupdf.readthedocs.io/en/latest/tutorial.html]

Cannot get PyMuPDF to work. The 'fitz' library will not load. The 'PyMuPDF' library will not load. 


In [ ]:
# !pip install PyMuPDF

In [ ]:
%run lib/__wesley_init_verbose__.py

In [ ]:
import sys, fitz

In [ ]:
# Opening a document
# doc = fitz.open(filename)

doc = fitz.open('data/schoolshooter/1997_1998_columbine_report.pdf')

print('Document object:', doc)
print('\n\nPage Count:', doc.pageCount)
print('\n\nMetadata:', doc.metadata)
# print('\n\nTable of Contents:', doc.getToC())
# print('Read a Page:', doc.loadPage())

In [ ]:
# Create a Page

page = doc.loadPage(12)  # loads page number 'pno' of the document (0-based)
# page = doc[1]  # the short form

print("\n\n'Page' variable value:", page)

# ... or even use 'slicing'
#for page in doc.pages(start, stop, step):
    # do something with 'page'
    
# get all links on a page
links = page.getLinks()

print("'Links' variable value:", links)

In [ ]:
# Rendering a Page as a picture

pix = page.getPixmap()

pix

In [ ]:
# Extracting Text and Images
# “text”: (default) plain text with line breaks. No formatting, no text position details, no images.
# “blocks”: generate a list of text blocks (= paragraphs).
# “words”: generate a list of words (strings not containing spaces).
# “html”: creates a full visual version of the page including any images. This can be displayed with your internet browser.
# “dict” / “json”: same information level as HTML, but provided as a Python dictionary or resp. JSON string. See TextPage.extractDICT() resp. TextPage.extractJSON() for details of its structure.
# “rawdict”: a super-set of TextPage.extractDICT(). It additionally provides character detail information like XML. See TextPage.extractRAWDICT() for details of its structure.
# “xhtml”: text information level as the TEXT version but includes images. Can also be displayed by internet browsers.
# “xml”: contains no images, but full position and font information down to each single text character. Use an XML module to interpret.

text = page.getText("text")

#Borrow two functions from wesley init
text = clean_the_text(text)
text = split_into_sentences(text)

text

In [ ]:
# Searching for text

areas = page.searchFor("general public", hit_max = 100)

for rec in areas:
    print(rec)

print(len(areas), '\n', areas)

In [ ]:
import sys
import fitz

def mark_word(page, text):
    """Underline each word that contains 'text'.
    """
    found = 0
    wlist = page.getTextWords()        # make the word list
    for w in wlist:                    # scan through all words on page
        if text in w[4]:               # w[4] is the word's string
            found += 1                 # count
            r = fitz.Rect(w[:4])       # make rect from word bbox
            page.addUnderlineAnnot(r)  # underline
    return found

fname = 'data/schoolshooter/1997_1998_columbine_report.pdf'# filename
text = 'general public'                     # search string
doc = fitz.open(fname)

print("underlining words containing '%s' in document '%s'" % (text, doc.name))

new_doc = False                        # indicator if anything found at all

for page in doc:                       # scan through the pages
    found = mark_word(page, text)      # mark the page's words
    if found:                          # if anything found ...
        new_doc = True
        print("found '%s' %i times on page %i" % (text, found, page.number + 1))

if new_doc:
    doc.save("marked-" + doc.name)

# Testing disparate string comparison

**Goal:** We want to extract the text from a PDF and compare that against a clean file of the same text in order to benchmark the PDF extraction techniques. 

We are comparing sentence to sentence to see where there are differences. 

In [ ]:
# !pip install python-Levenshtein

In [ ]:
# Jeanna's Code

import pandas as pd
%run lib/__wesley_init__.py

def pdf_to_txt(path):
    #from pdfminer.six documentation: https://pdfminersix.readthedocs.io/en/latest/tutorial/composable.html
    from io import StringIO
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfdocument import PDFDocument
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.pdfparser import PDFParser
    output_string = StringIO()
    with open(path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    text = str(output_string.getvalue())
    return text

In [ ]:
# pdfminer.six from Jeanna

from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.high_level import *


output_string = StringIO()
with open('data/research_articles/689031.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue())


In [ ]:
# '''Following two strings have two differences between them. Determining techniques to discern and 
# show where the differences are located. '''


# string_1 = "Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal. \n \
#             Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come \
#             to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this. \n \
#             But, in a larger sense, we can not dedicate -- we can not consecrate -- we can not hallow -- this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our \
#             poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here \
#             to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us -- that from these honored dead \
#             we take increased devotion to that cause for which they gave the last full measure of devotion -- that we here highly resolve that these dead shall not have died in vain -- that this nation, under \
#             God, shall have a new birth of freedom -- and that government of the people, by the people, for the people, shall not perish from the earth."

# string_2 = "Four score and seven years ag o our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal. \n \
#             Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come \
#             to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this. \n \
#             But, in a larger sense, we can not dedicate -- we can not consecrate -- we can not hallow -- this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our \
#             poor power to add or detract. The world will little note, norlongremember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here \
#             to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us -- that from these honored dead \
#             we take increased devotion to that cause for which they gave the last full measure of devotion -- that we here highly resolve that these dead shall not have died in vain -- that this nation, under \
#             God, shall have a new birth of freedom -- and that government of the people, by the people, for the people, shall not perish from the earth."

In [ ]:
%run lib/__wesley_init__.py

string_1 = pdf_to_txt('data/research_articles/689031.pdf')
string_1 = clean_the_text(string_1)
string_1 = string_1.replace('1 ', '')
string_1 = string_1.replace('2 ', '')
string_1 = string_1.replace('3 ', '')

In [ ]:
# with open('data.txt', 'r') as file:
#     data = file.read().replace('\n', '')
string_2 = txt_to_txt('data/research_articles/689031.txt')
string_2 = clean_the_text(string_2)
string_2 = string_2.replace('1 ', '')
string_2 = string_2.replace('2 ', '')
string_2 = string_2.replace('3 ', '')

## Testing Levenshtein distance

In [ ]:
import Levenshtein

In [ ]:
Levenshtein.distance(string_1, string_2)

So the result of 333 means that it would take 333 transformations (of one type or another) to get the two strings to be the same. This could serve as a means to benchmark the pdf extraction transformations. 

## Cosine Similarity

In [ ]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

def clean_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

corpi = [string_1, string_2]

corpi = list(map(clean_string, corpi))

vectorizer = CountVectorizer().fit_transform(corpi)
vectors = vectorizer.toarray()

cosim = cosine_similarity(vectors)

def cosine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

cosine_sim_vectors(vectors[0], vectors[1])

In [ ]:
string_2

In [ ]:
#Straight comparison of the two strings. 
print(string_1 == string_2)

In [ ]:
# Comparison of lengths
print('length string_1:', len(string_1), '\nlength string_2:', len(string_2))
print(len(string_1) == len(string_2))

In [ ]:
# Find the sentences where there is a difference

string_1_sentences = string_1.split('. ')
string_2_sentences = string_2.split('. ')

print('length of string_1 =', len(string_1_sentences))
print('length of string_2 =', len(string_2_sentences))

if len(string_1_sentences) > len(string_2_sentences):
    smallest = len(string_2_sentences)
elif len(string_1_sentences) < len(string_2_sentences):
    smallest = len(string_1_sentences)
else:
    smallest = len(string_1_sentences)

for counter in range(0, smallest):
    if (len(string_1_sentences[counter]) != len(string_2_sentences[counter])):
        print('sentences', counter, 'are different')
        
        string_1_words = string_1_sentences[counter].split(' ')
        string_2_words = string_2_sentences[counter].split(' ')
        
        print(len(string_1_words), len(string_2_words))
        
        if len(string_1_words) > len(string_2_words):
            words_counter = len(string_1_words)
        else:
            words_counter = len(string_2_words)
            
        print("the greater sentence word count is", words_counter)
        
        for i in range(words_counter):
            if string_1_words[i] == string_2_words[i]:
                print('word', i, 'is the same')
            else:
                print('difference at word', i, '. Sentence 1 word:',string_1_words[i], 'Sentence 2 word:',string_2_words[i])  
                break
    else: print('sentences are the same')

## TF-IDF Similarity Test

coded by Eric Naon


In [ ]:
def tfidf_string_similarity_test(correct_string,test_string):
    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    two_strings = [correct_string,test_string]
    vect = TfidfVectorizer(min_df=1, stop_words="english")
    tfidf = vect.fit_transform(two_strings)
    pairwise_similarity = tfidf * tfidf.T
    arr = pairwise_similarity.toarray()
    np.fill_diagonal(arr, -1)
    return arr[0][1]

In [ ]:
%%time

tfidf_string_similarity_test(string_2, string_1)

# Manage document same content

In [ ]:
def manage_same_document_content():
    
    
    '''
    1. Prompt user to enter the directory that contains files to check
    2. If same file found, ask user which one is the most recent file (to keep)
    3. Ask user to choose either delete the old file or move it to "temp" directory
    
    '''
    
    
    import os
    from colorama import Fore, Back

    while True:
        try:
            input_directory_name = input("\nEnter the directory name: ")
            filenames = os.listdir(input_directory_name)
        except:
            print("Directory not found!")
        else:
            break

    def ask_recent(file1, file2):
        print("\n\n")
        print(Fore.RED + file1 + Fore.RESET + " and " + Fore.BLUE + file2 + Fore.RESET + " have the same content.\n")
        while True:
            print(Fore.RED + f"1. {file1}")
            print(Fore.BLUE + f"2. {file2}" + Fore.RESET)
            try:
                result = int(input("\nWhich one is the most recent file? (1/2): "))
            except:
                print("Invalid input! Please try again\n\n")
            else:
                if result == 1 or result == 2:
                    break
                else:
                    print("Invalid input! Please try again\n\n")

        return result

    def delete_or_move(old_file_name, recent_choice):

        # coloring
        if recent_choice == 1:
            color_choice = Fore.BLUE
        else:
            color_choice = Fore.RED

        while True:
            print("\nWhat to do with " + color_choice + old_file_name + Fore.RESET + "?")
            print(f"1. Move to temp directory")
            print(f"2. Delete")
            try:
                result = int(input("Press (1/2): "))
            except:
                print("Invalid input! Please try again\n")
            else:
                if result == 1 or result == 2:
                    break
                else:
                    print("Invalid input! Please try again\n")


        if result == 1: 

            # create temp directory if it doesn't exist
            if not os.path.exists("temp"):
                os.makedirs("temp")

            # move the file to temp dir
            os.replace(os.path.join(input_directory_name, old_file_name), os.path.join("temp", old_file_name))
            print("Successfully move " + color_choice + old_file_name + Fore.RESET + " to temp directory")

        else:

            # delete file
            os.remove(os.path.join(input_directory_name, old_file_name))
            print("Successfully remove " + color_choice + old_file_name + Fore.RESET)

        return 

    def read_file(file):
        with open(os.path.join(input_directory_name, file), "rb") as f:
            result = f.read()
        return result

    
    # checking starts here
    i1 = 0
    while True:

        if i1 == len(filenames):
            break

        filename_1 = filenames[i1]
        result_1 = read_file(filename_1)
        i2 = i1 + 1
        while True:

            if i2 == len(filenames):
                break

            filename_2 = filenames[i2]
            result_2 = read_file(filename_2)

            if result_1 == result_2:

                recent_choice = ask_recent(filename_1, filename_2)
                if recent_choice == 1:

                    delete_or_move(filename_2, recent_choice)
                    filenames.remove(filename_2)

                else:

                    delete_or_move(filename_1, recent_choice)
                    filenames.remove(filename_1)
                    i1 -= 1
                    break
            else:
                i2 += 1

        i1 += 1
        
    print("\n\nChecking done!")

In [ ]:
manage_same_document_content()

# Second Search Code with PyMuPDF

In [21]:
%run lib/__wesley_init_verbose__.py

import fitz
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")


	Welcome to the Wesley Initiation File! 

        
The Wesley Initiation File consists of multiple functions to help us with our tasks. 
I am loading the following functions:
	find_file(pathway)
	walk_directory(directory)
	walk_directory_verbose(directory)
	initialize_data_log()
	initialize_syn_log()
	initialize_url_log()
	pdf_to_txt(path)
	txt_to_txt(path)
	docx_to_txt(path)
	rtf_to_txt(path)
	to_string(text)
	to_bytes(text)
	clean_the_text(text, remove_numbers=False)
	pos_tag(text)
	count_unique_words(string)
	count_words(text)
	count_word_frequency(text)
	determine_pdf_encoding(file)
	ngram_list(text, n_gram_size)
	find_keywords(text)
	tokenize_by_words(text)
	tokenize_by_words2(text)
	split_into_sentences(text)
	simple_tokenize_by_sentences(text)
	tokenize_by_paragraphs(text)
	remove_stopwords(text, is_lower_case=False)
	simple_stemmer(text)
	lemmatize_text(text)
	single_word_search(search_word)
	accessed_created_modified(file)
	plot_top_ngrams_barchart(text, n=2)
	plot_named_entit

In [22]:
# Walk a directory of files and apply basic PyMuPDF functions to files ending with .pdf
# This is just a test of code and does not apply to much. 


def search_directory_for_term(directory):
    search_term = input("Search For: ")
    search_term = ' '+ search_term +' '
    limiter = input("How many files would you like me to search? ")
    limiter = limiter.lower()
    
    if limiter == 'all':
        for root, dirs, files in os.walk(directory):
            limiter = int(len(files))
            print('\nThere are', limiter, 'files in this directory.')
            verify = input('Shall I continue?')
            verify = verify.lower()
            if verify != 'yes':
                return
        
    else:
        limiter = int(limiter)-1
    
    for root, dirs, files in os.walk(directory):
#         print(len(files))
        if len(files) < limiter:
            print("We have", len(files), "files in the directory.")
            limiter = input("How many files would you like me to search? ")
            if limiter == 'all':
                limiter = int(len(df))
            else:
                limiter = int(limiter)-1
            
    counter = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if counter > limiter: # how many files should we look at?
                break
            elif file.startswith('.'):
                pass
            elif file.endswith('.pdf'):
                pathway = os.path.join(root, file)
                pathway = os.path.realpath(pathway)# added from Jeanna for Windows to read
                
                counter += 1 #This is just to limit the number of files I am reading for this test
                
                def get_number_of_search_instances_pymupdf(pathway, search_term):
                    import fitz
                    doc = fitz.open(pathway)
                    areas_append = 0
                    for page_num in range(0, doc.pageCount):
                        page = doc.loadPage(page_num)
#                         text = page.getText("text")

                        #Borrow two functions from wesley init
#                         text = clean_the_text(text)
#                         text = split_into_sentences(text)

                        areas_num = 0
                        areas_lower = 0
                        areas_upper = 0
                        areas_title = 0

                        areas_lower = page.searchFor(search_term.lower(), hit_max = 100)
                        areas_upper = page.searchFor(search_term.upper(), hit_max = 100)
                        areas_title = page.searchFor(search_term.title(), hit_max = 100)

                        areas_num = int((len(areas_lower) + len(areas_upper) + len(areas_title))/3)


                        if areas_num > 0:
                            areas_append = areas_append + areas_num

                    return areas_append, doc
    
                
                temp_count, doc = get_number_of_search_instances_pymupdf(pathway, search_term)
                print('\n', '*'*82, "\nThe search term,", search_term +', appears', temp_count,'times in', doc,'.')
    
                if temp_count > 0:
                    for page_num in range(0, doc.pageCount):
                        page = doc.loadPage(page_num)
                        text = page.getText("text")

                        #Borrow two functions from wesley init
#                         text = clean_the_text(text)
                        text = split_into_sentences(text)
                        
                        areas_lower = page.searchFor(search_term.lower(), hit_max = 100)
                        areas_upper = page.searchFor(search_term.upper(), hit_max = 100)
                        areas_title = page.searchFor(search_term.title(), hit_max = 100)
                        for sentence in text:
                            sentence = clean_the_text(sentence)
                            
                            if search_term.lower() in sentence:
                                print('\n', '*'*82, '\nThe search term,'+ search_term + 'appears on page '+  str(page_num) + '.')
                                print((sentence.replace(search_term.lower(), '\033[43;30m{}\033[m'.format(search_term))), '\n\n')
                            elif search_term.upper() in sentence:
                                print('\n', '*'*82, '\nThe search term,'+ search_term + 'appears on page '+  str(page_num) + '.')
                                print((sentence.replace(search_term.upper(), '\033[43;30m{}\033[m'.format(search_term))), '\n\n')
                            elif search_term.title() in sentence:
                                print('\n', '*'*82, '\nThe search term,'+ search_term + 'appears on page '+  str(page_num) + '.')
                                print((sentence.replace(search_term.title(), '\033[43;30m{}\033[m'.format(search_term))), '\n\n')


                        else: 
                            pass
            else:
                pass
    print('------- END OF SEARCH -------')
    
which_topic = input('What topic would you like me to search?')
which_topic = which_topic.rstrip()
which_topic = which_topic.replace(' ', '_')

directory = which_topic + '_data'
search_directory_for_term(directory)

What topic would you like me to search?human trafficking
Search For: girls
How many files would you like me to search? all

There are 11 files in this directory.
Shall I continue?yes

 ********************************************************************************** 
The search term,  girls , appears 0 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/Thehardtruthabouthumantrafficking.pdf') .

 ********************************************************************************** 
The search term,  girls , appears 75 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/index.pdf') .

 ********************************************************************************** 
The search term, girls appears on page 4.
Notwithstanding these general data, there is no clear consensus on the numbers of girls versus boys exploited through prostitution nationwide. 



 **************************************************************************


 ********************************************************************************** 
The search term, girls appears on page 25.
YWEP employs adolescent girls who have exited prostitution or are still engaged in prostitution and are willing to attend a 16-session training to conduct outreach and education to peers. 



 ********************************************************************************** 
The search term, girls appears on page 25.
The girls receive a stipend for their time. 



 ********************************************************************************** 
The search term, girls appears on page 25.
The Young Womens Leadership Project in Detroit similarly employs girls with a history of some type of high-risk activity to provide street outreach to their at-risk peers (Alternatives for Girls, 2006). 



 ********************************************************************************** 
The search term, girls appears on page 29.
Briceno, personal communication, June 20


 ********************************************************************************** 
The search term, girls appears on page 123.
A study of the circumstances in which girls migrated and were trafficked into the commercial sex industry within one South-East Asian country identified the social and cultural reasons why certain adolescents were recruited and not others. 



 ********************************************************************************** 
The search term, girls appears on page 125.
They include films telling semi-fictional stories about adolescent girls or young women who have been trafficked into commercial sexual exploitation. 



 ********************************************************************************** 
The search term, girls appears on page 128.
In the case of the United Nations-administered Province of Kosovo, a law containing this provision was intended to address demand for women and girls trafficked into prostitution. 



 *****************************


 ********************************************************************************** 
The search term, girls appears on page 57.
Detention of women and girls and restrictions on the emigration of women and girls are just two examples of potentially discriminatory responses to traffcking. 



 ********************************************************************************** 
The search term,  girls , appears 0 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/Crime Victim Rights PDF.pdf') .

 ********************************************************************************** 
The search term,  girls , appears 5 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/HumanTrafficking Report.pdf') .

 ********************************************************************************** 
The search term, girls appears on page 5.
The Mann Act made it illegal to transport women or girls across state lines for the purposes of sexual act


 ********************************************************************************** 
The search term, girls appears on page 116.
The government and media reported cases of girls younger than 18 working in drayangs, a violation of the minimum age requirement of 18. 



 ********************************************************************************** 
The search term, girls appears on page 116.
Trafickers have exploited Bhutanese women and girls working in domestic labor, caregiving, and in sex and labor traficking, including through debt bondage and threats of physical abuse. 



 ********************************************************************************** 
The search term, girls appears on page 116.
Bhutanese women and girls who work as entertainers in drayangs are vulnerable to labor and sex trafickers. 



 ********************************************************************************** 
The search term, girls appears on page 119.
Bolivian women and girls are exploited in 

...

 ********************************************************************************** 
The search term, girls appears on page 367.
Although the incidence of child domestic workers has reportedly decreased in Morocco since 2005, girls are recruited from rural areas for work in domestic service in cities and some become victims of forced labor. 



 ********************************************************************************** 
The search term, girls appears on page 367.
NGOs and other observers anecdotally reported in 2018 that a significant number of girls work as domestic help in Moroccan households, but it is dificult to determine the extent of the problem because of authorities inability to access this population. 



 ********************************************************************************** 
The search term, girls appears on page 369.
Trafickers lure voluntary migrants, especially women and girls from rural areas, from neighboring countries to cities in Mozambique or Sou


 ********************************************************************************** 
The search term, girls appears on page 401.
Some Pakistani trafickers lure women and girls away from their families with promises of marriage, create fraudulent marriage certificates, and exploit the women and girls in sex traficking, including in Iran and Afghanistan. 



 ********************************************************************************** 
The search term, girls appears on page 401.
In other cases, trafickers, including some extra-judicial courts, use girls as chattel to settle debts or disputes. 



 ********************************************************************************** 
The search term, girls appears on page 401.
Trafickers have exploited Pakistani girls in sex traficking in Kenya and Pakistani adults, including with disabilities, to forced begging in the United Arab Emirates. 



 ********************************************************************************** 
The se


 ********************************************************************************** 
The search term, girls appears on page 460.
Under informal justice practices referred to as customary rules, parents frequently receive payments for sending young women and girls into forced marriages with foreign workers at logging and mining companies, where many of them are exploited in domestic servitude or sex traficking. 



 ********************************************************************************** 
The search term, girls appears on page 460.
Mining and logging camp leadership reportedly force boys to serve as solairsillicit brokers procuring girls for sexual and domestic service in worker lodging facilitiesand logging camp personnel force young males to work as domestic servants and cooks. 



 ********************************************************************************** 
The search term, girls appears on page 464.
Male occupants of the household sexually abuse some of these women


 ********************************************************************************** 
The search term, girls appears on page 499.
Illicit networks exploit Ghanaian girls in sex traficking in Togo. 



 ********************************************************************************** 
The search term, girls appears on page 501.
Trafickers lure women and girls from Colombia, the Dominican Republic, and Venezuela with ofers of employment, many via social media, and subject them to sex traficking in brothels and clubs. 



 ********************************************************************************** 
The search term, girls appears on page 501.
Trafickers are increasingly targeting and accompanying vulnerable foreign young women and girls between the ages of 15 and 21. 



 ********************************************************************************** 
The search term, girls appears on page 504.
Some Tunisian girls working as domestic servants for wealthy families in Tunis and ma


 ********************************************************************************** 
The search term, girls appears on page 546.
Trafickers exploit women and girls from Zimbabwean towns bordering South Africa, Mozambique, and Zambia in forced labor, including domestic servitude, and sex traficking in brothels catering to long-distance truck drivers on both sides of the border. 



 ********************************************************************************** 
The search term, girls appears on page 548.
The government did not have any policy structures, institutional capacity, widespread political will, or resources to proactively identify and protect traficking victims among vulnerable groups, such as foreign migrants, refugees, and asylum-seekers; women and girls in commercial sex; and children recruited and used by government-aligned militias or other armed groups. 



 ********************************************************************************** 
The search term, girls ap

# Search Method Testing


In [24]:
# Method 1: Using Python's in function and tokenized sentences to find search terms in a document

def get_number_of_search_instances_in(pathway, search_term):
    import fitz
    
    files = walk_directory(pathway)
    
    for file in files:
        
        doc = fitz.open(file)

        temp_count = 0

        for page_num in range(0, doc.pageCount):
            page = doc.loadPage(page_num)
            text = page.getText("text")

            #Borrow two functions from wesley init
            text = clean_the_text(text)
            text = simple_tokenize_by_sentences(text)


            for sentence in text:
                if search_term.lower() in sentence:
                    temp_count += 1
                elif search_term.upper() in sentence:
                    temp_count += 1
                elif search_term.title() in sentence:
                    temp_count += 1


        if temp_count > 0:
            print('\n', '*'*82, "\nThe search term,", search_term +', appears', temp_count,'times in', doc,'.')


#     return temp_count, doc

which_topic = input('What topic would you like me to search?')
which_topic = which_topic.rstrip()
which_topic = which_topic.replace(' ', '_')

directory = which_topic + '_data'

search_term = input('What is our search term?')
get_number_of_search_instances_in(directory, search_term)


What topic would you like me to search?human trafficking
What is our search term?prostitute


1it [00:00, 966.21it/s]



 ********************************************************************************** 
The search term, prostitute, appears 35 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/index.pdf') .

 ********************************************************************************** 
The search term, prostitute, appears 7 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/An_Introduction_to_Human_Trafficking_-_Background_Paper.pdf') .

 ********************************************************************************** 
The search term, prostitute, appears 10 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/HumanTrafficking Report.pdf') .

 ********************************************************************************** 
The search term, prostitute, appears 2 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/2020-TIP-Report-Complete-062420-FINAL.pdf') .

 ********

In [25]:
# Method 2: Using PyMuPDF's .searchFor() function to find search terms in a document


def get_number_of_search_instances_pymupdf(pathway_to_file, search_term):
    import fitz
    doc = fitz.open(pathway_to_file)
    areas_append = 0
    for page_num in range(0, doc.pageCount):
        page = doc.loadPage(page_num)
        text = page.getText("text")

        #Borrow two functions from wesley init
        text = clean_the_text(text)
        text = simple_tokenize_by_sentences(text)

        areas_num = 0
        areas_lower = 0
        areas_upper = 0
        areas_title = 0
        
        areas_lower = page.searchFor(search_term.lower(), hit_max = 100)
        areas_upper = page.searchFor(search_term.upper(), hit_max = 100)
        areas_title = page.searchFor(search_term.title(), hit_max = 100)
        
        areas_num = int((len(areas_lower) + len(areas_upper) + len(areas_title))/3)
        
        
#         if areas_num > 0:
#             print('\n', '*'*82, "\nThe search term,", search_term +', appears', areas_num,'times on page', str(page_num),'in', doc,'.')

        areas_append = areas_append + areas_num
            
    return areas_append

which_topic = input('What topic would you like me to search?')
which_topic = which_topic.rstrip()
which_topic = which_topic.replace(' ', '_')

directory = which_topic + '_data'

search_term = input('What is our search term?')
print(get_number_of_search_instances_in(directory, search_term))

number = get_number_of_search_instances_pymupdf(directory, search_term)
print('\n', '*'*82, "\nMethod 2:\nThe search term,", search_term +', appears', number,'times in', pathway,'.')

What topic would you like me to search?human trafficking
What is our search term?prostitute


1it [00:00, 924.06it/s]



 ********************************************************************************** 
The search term, prostitute, appears 35 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/index.pdf') .

 ********************************************************************************** 
The search term, prostitute, appears 7 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/An_Introduction_to_Human_Trafficking_-_Background_Paper.pdf') .

 ********************************************************************************** 
The search term, prostitute, appears 10 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/HumanTrafficking Report.pdf') .

 ********************************************************************************** 
The search term, prostitute, appears 2 times in Document('/Users/benmeredith/Desktop/GitHub/Wesley/human_trafficking_data/2020-TIP-Report-Complete-062420-FINAL.pdf') .

 ********

mupdf: read error: Is a directory
mupdf: cannot recognize version marker
mupdf: read error: Is a directory
mupdf: read error: Is a directory
mupdf: cannot tell in file


RuntimeError: cannot tell in file

In [26]:
df

,root,file,file_path,file_type,raw_text,clean_text,word_token,key_words,urls,emails,...,created,last_modified,word_count,proper_names,nouns,verbs,adjectives,adverbs,possessives,pronouns
0,human_trafficking_data,Thehardtruthabouthumantrafficking.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,"b'See discussions, stats, and author profiles ...","bSee discussions, stats, and author profiles f...","['bSee', 'discussions,', 'stats,', 'and', 'aut...","victim, victimization, trafficked, traffickers...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:24 2020,3496.0,"{'persons', 'analysis', 'j', 'thus', 'forensic...","{'time', 'adherence', 'price', 'someone', 'ind...","{'overcrowding', 'approved', 'deport', 'involv...","{'certain', 'underground', 'empirical', 'vital...","{'further', 'always', 'as', 'less', 'clearly',...","{'their', 'his', 'its', 'your', 'our', 'her'}","{'they', 'she', 'we', 'it', 'them', 'i', 'he',..."
1,human_trafficking_data,index.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'U.S. Department of Health and Human Services...,"bU.S. Department of Health and Human Services,...","['bU.S.', 'Department', 'of', 'Health', 'and',...","services, trafficking, trafficked, traffickers...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:58 2020,29771.0,"{'comparative', 'appropriate', 'gordon', 'thai...","{'time', 'advantage', 'citations', 'exacerbati...","{'deliver', 'uncounted', 'appropriate', 'assum...","{'certain', '47-54', 'www.protectionproject.or...","{'al.', 'later', 'errata', 'while', 'always', ...","{'their', 'his', 'its', 'our', 'her'}","{'we', 'they', 'she', 'itself', 'it', 'him', '..."
2,human_trafficking_data,An_Introduction_to_Human_Trafficking_-_Backgro...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,"b'Vienna International Centre, PO Box 500, 140...","bVienna International Centre, PO Box 500, 1400...","['bVienna', 'International', 'Centre,', 'PO', ...","trafficking, trafficked, traffickers, traffick...",NaN,['ahtu@unodc.org'],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:02 2020,135190.0,"{'comparative', 'overseas', 'x93n', 'mechanism...","{'time', 'beneficiaries', 'xa8les', 'relaciona...","{'appropriate', 'relacionadas', 'mentioned', '...","{'appropriate', 'non-formal', 'inter-ministeri...",{'www.ilo.org/dyn/declaris/declarationweb.down...,"{'their', 'his', 'its', 'my', 'our', 'her'}","{'they', 'we', 'herself', 'itself', 'she', 'x9..."
3,human_trafficking_data,bc-infosheet-human-trafficking-101-english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'Human Trafficking 101\nWhat is Human Traffic...,bHuman Trafficking 101 What is Human Trafficki...,"['bHuman', 'Trafficking', '101', 'What', 'is',...","victims, victim, trafficking, trafficked, traf...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:51 2020,635.0,"{'customs', 'english', 'enforcement', 'tip-lin...","{'care', 'sight', 'www.ice.gov/tips', 'job', '...","{'help', 'pay', 'involves', 'hidden', 'engaged...","{'anxious', 'false', 'human', 'long', 'and/or'...","{'necessarily', 'freely', 'instead', 'also', '...","{'their', 'her'}","{'they', 'him', 'us', 'i', 'you'}"
4,human_trafficking_data,WHO_RHR_12.42_eng.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'Understanding and addressing\nviolence again...,bUnderstanding and addressing violence against...,"['bUnderstanding', 'and', 'addressing', 'viole...","trafficked, traffickers, health, human traffic...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:16 2020,3129.0,"{'empirical', 'x9335', 'detention', 'thailand'...","{'time', 'overview', 'exposure', 'figures', 'c...","{'varying', 'assess', 'has', 'does', 'persiste...","{'appropriate', 'referral', 'airborne', 'signi...","{'immediately', 'even', 'again', 'approximatel...","{'its', 'their'}","{'we', 'they', 'it', 'he', 'i'}"
5,human_trafficking_data,FS36_en.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'Human Rights\nand Human Trafficking\n\nFact ...,bHuman Rights and Human Tra

# Find word frequency in documents we have

Find the word frequency in documents that we have that are presorted by type of document in order to find common words to later use to sort documents by type through Python:


In [28]:
import os
import pandas as pd
from tqdm import tqdm

topic = input('What topic are we searching today?')
topic = topic.rstrip()
topic = topic.replace(' ', '_')

df_pathway = topic + '_data_table.csv'

df = pd.read_csv(df_pathway, index_col=0)

df


What topic are we searching today?human trafficking


,root,file,file_path,file_type,raw_text,clean_text,word_token,key_words,urls,emails,...,created,last_modified,word_count,proper_names,nouns,verbs,adjectives,adverbs,possessives,pronouns
0,human_trafficking_data,Thehardtruthabouthumantrafficking.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,"b'See discussions, stats, and author profiles ...","bSee discussions, stats, and author profiles f...","['bSee', 'discussions,', 'stats,', 'and', 'aut...","victim, victimization, trafficked, traffickers...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:48:24 2020,3496.0,"{'persons', 'analysis', 'j', 'thus', 'forensic...","{'time', 'adherence', 'price', 'someone', 'ind...","{'overcrowding', 'approved', 'deport', 'involv...","{'certain', 'underground', 'empirical', 'vital...","{'further', 'always', 'as', 'less', 'clearly',...","{'their', 'his', 'its', 'your', 'our', 'her'}","{'they', 'she', 'we', 'it', 'them', 'i', 'he',..."
1,human_trafficking_data,index.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'U.S. Department of Health and Human Services...,"bU.S. Department of Health and Human Services,...","['bU.S.', 'Department', 'of', 'Health', 'and',...","services, trafficking, trafficked, traffickers...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:47:58 2020,29771.0,"{'comparative', 'appropriate', 'gordon', 'thai...","{'time', 'advantage', 'citations', 'exacerbati...","{'deliver', 'uncounted', 'appropriate', 'assum...","{'certain', '47-54', 'www.protectionproject.or...","{'al.', 'later', 'errata', 'while', 'always', ...","{'their', 'his', 'its', 'our', 'her'}","{'we', 'they', 'she', 'itself', 'it', 'him', '..."
2,human_trafficking_data,An_Introduction_to_Human_Trafficking_-_Backgro...,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,"b'Vienna International Centre, PO Box 500, 140...","bVienna International Centre, PO Box 500, 1400...","['bVienna', 'International', 'Centre,', 'PO', ...","trafficking, trafficked, traffickers, traffick...",NaN,['ahtu@unodc.org'],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:02 2020,135190.0,"{'comparative', 'overseas', 'x93n', 'mechanism...","{'time', 'beneficiaries', 'xa8les', 'relaciona...","{'appropriate', 'relacionadas', 'mentioned', '...","{'appropriate', 'non-formal', 'inter-ministeri...",{'www.ilo.org/dyn/declaris/declarationweb.down...,"{'their', 'his', 'its', 'my', 'our', 'her'}","{'they', 'we', 'herself', 'itself', 'she', 'x9..."
3,human_trafficking_data,bc-infosheet-human-trafficking-101-english.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'Human Trafficking 101\nWhat is Human Traffic...,bHuman Trafficking 101 What is Human Trafficki...,"['bHuman', 'Trafficking', '101', 'What', 'is',...","victims, victim, trafficking, trafficked, traf...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:51 2020,635.0,"{'customs', 'english', 'enforcement', 'tip-lin...","{'care', 'sight', 'www.ice.gov/tips', 'job', '...","{'help', 'pay', 'involves', 'hidden', 'engaged...","{'anxious', 'false', 'human', 'long', 'and/or'...","{'necessarily', 'freely', 'instead', 'also', '...","{'their', 'her'}","{'they', 'him', 'us', 'i', 'you'}"
4,human_trafficking_data,WHO_RHR_12.42_eng.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'Understanding and addressing\nviolence again...,bUnderstanding and addressing violence against...,"['bUnderstanding', 'and', 'addressing', 'viole...","trafficked, traffickers, health, human traffic...",NaN,[],...,Wed Nov 18 05:15:04 2020,Wed Nov 18 04:45:16 2020,3129.0,"{'empirical', 'x9335', 'detention', 'thailand'...","{'time', 'overview', 'exposure', 'figures', 'c...","{'varying', 'assess', 'has', 'does', 'persiste...","{'appropriate', 'referral', 'airborne', 'signi...","{'immediately', 'even', 'again', 'approximatel...","{'its', 'their'}","{'we', 'they', 'it', 'he', 'i'}"
5,human_trafficking_data,FS36_en.pdf,/Users/benmeredith/Desktop/GitHub/Wesley/human...,pdf,b'Human Rights\nand Human Trafficking\n\nFact ...,bHuman Rights and Human Tra

# .docx scanning

https://stackoverflow.com/questions/55718448/how-do-i-get-docx2txt-to-process-all-docx-files-in-directory

https://www.analyticsvidhya.com/blog/2017/03/read-commonly-used-formats-using-python/



In [ ]:
# !pip install docx2txt

In [ ]:
import docx2txt
import os
%run lib/__wesley_init_verbose__.py

In [ ]:
iterator = 0
file_list = []

path = ('../') # Path to Desktop

# path = ('/Volumes/ONEDRIVE')


for root, dirs, files in tqdm(os.walk(path)):
    for file in files:
        if file.startswith('.'):
            pass
        elif file.endswith('.docx'):
            print(root, file)
#             pass
#         else:
            
#                 df.loc[iterator, 'root'] = root
#                 df.loc[iterator, 'file'] = file
#             print('len(files):', len(files))
#             print('root:', root)
#             print('dirs:', dirs)
#             print('files:', files)
            pathway = os.path.join(root, file)
            pathway = os.path.realpath(pathway)# added from Jeanna for Windows to read
            file_list.append(pathway)
            iterator += 1

In [ ]:
file_list

# Newspaper & TextBlob Tests

In [ ]:
%run lib/__wesley_init_verbose__.py

In [ ]:
import pandas as pd
from newspaper import Article

import warnings
warnings.filterwarnings("ignore")

import logging
logger = logging.getLogger()
logger.propagate = False

df = pd.read_csv('url_results.csv', index_col=[0])

df.head()

In [ ]:
for i, url in enumerate(df['url']):
    article = Article(url)
    article.download()
    article.parse()
    
    article_authors = article.authors
    print(article_authors)
    
#     df.loc[i, 'authors'] = article_authors